# 📋 CRISP-DM Framework for stock market chart pattern recognition using deep learning 

## Phase 1: Business Understanding

1. Business Understanding
วัตถุประสงค์: เพื่อสร้างระบบพยากรณ์ราคาและตรวจจับสัญญาณทางเทคนิคสำหรับหุ้น/คริปโต (เช่น BTC) โดยใช้การผสมผสานระหว่าง:

LSTM: พยากรณ์แนวโน้มราคาส่วนต่าง (Time-series)

CNN: ตรวจจับรูปแบบกราฟ Head & Shoulders (Pattern Recognition)

NLP: วิเคราะห์อารมณ์ตลาดจากข่าว (Sentiment Analysis)

### Cell 1: Preparation 
รวม Library ทั้งหมดที่ต้องใช้ในงานวิจัย

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from cassandra.cluster import Cluster
from transformers import pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score

# TensorFlow / Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

### 2.data understanding 

In [48]:
def get_data(symbol):
    cluster = Cluster([CASSANDRA_HOST], port=CASSANDRA_PORT)
    session = cluster.connect('data_stock')
    query = "SELECT time, open, high, low, close, volume FROM candlestick_data WHERE symbol=%s LIMIT 1000 ALLOW FILTERING"
    df = pd.DataFrame(list(session.execute(query, (symbol,))))
    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values('time').reset_index(drop=True)
    cluster.shutdown()
    return df

df_raw = get_data(SYMBOL)
print(f"✅ ข้อมูลถูกโหลดเรียบร้อย: {len(df_raw)} แถว")
df_raw.head()

✅ ข้อมูลถูกโหลดเรียบร้อย: 729 แถว


,time,open,high,low,close,volume
0,2023-01-17,12.4,12.4,12.2,12.4,49432995
1,2023-01-18,12.4,12.5,12.2,12.4,68819770
2,2023-01-19,12.3,12.4,12.2,12.3,77191342
3,2023-01-20,12.4,12.7,12.3,12.7,193937283
4,2023-01-23,12.6,12.6,12.4,12.5,57961993


### Cell 3: Data Preparation & Labeling
สร้าง Label สำหรับรูปทรง H&S และทำ Scaling (Normalization)

In [49]:
def get_hs_labels(df, window=5):
    prices = df['close'].values
    labels = np.zeros(len(prices))
    peaks = [i for i in range(window, len(prices)-window) if all(prices[i] > prices[i-j] and prices[i] > prices[i+j] for j in range(1, window+1))]
    patterns = []
    for i in range(len(peaks)-2):
        ls, h, rs = peaks[i], peaks[i+1], peaks[i+2]
        if prices[h] > prices[ls]*1.03 and prices[h] > prices[rs]*1.03 and abs(prices[ls]-prices[rs])/prices[ls] < 0.12:
            labels[ls-5:rs+6] = 1
            lv, rv = ls + np.argmin(prices[ls:h+1]), h + np.argmin(prices[h:rs+1])
            patterns.append({'ls': (ls, prices[ls]), 'h': (h, prices[h]), 'rs': (rs, prices[rs]), 'neck': [(lv, prices[lv]), (rv, prices[rv])]})
    return labels, patterns

labels, patterns_found = get_hs_labels(df_raw)

# Scaling สำหรับ LSTM (ราคา)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df_raw[['high', 'low', 'close']])

# ฟังก์ชันแปลงราคาจาก 0-1 กลับเป็นบาท
def inv(v): return scaler.inverse_transform(np.column_stack([np.zeros((len(v), 2)), v]))[:, 2]

X_l, y_l = [], []
for i in range(len(scaled_data)-SEQ_LEN):
    X_l.append(scaled_data[i:i+SEQ_LEN])
    y_l.append(scaled_data[i+SEQ_LEN, 2])

X_l, y_l = np.array(X_l), np.array(y_l)
split = int(len(X_l) * 0.8)
print(f"✅ เตรียมข้อมูลสำเร็จ: พบ Pattern ทั้งหมด {len(patterns_found)} จุด")

✅ เตรียมข้อมูลสำเร็จ: พบ Pattern ทั้งหมด 3 จุด


### Cell 4: Modeling (LSTM & CNN)
สร้างและเทรนโมเดลสำหรับการทำนายแนวโน้มและตรวจจับรูปทรง

In [50]:
# 1. LSTM (Trend Prediction)
model_lstm = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 3)),
    LSTM(32),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_l[:split], y_l[:split], epochs=20, batch_size=32, verbose=0, callbacks=[EarlyStopping(patience=3)])

# 2. CNN (Pattern Recognition)
X_c, y_c = [], []
scaled_cnn = MinMaxScaler().fit_transform(df_raw[['close', 'volume']])
for i in range(len(scaled_cnn)-SEQ_LEN):
    X_c.append(scaled_cnn[i:i+SEQ_LEN])
    y_c.append(labels[i+SEQ_LEN])
X_c, y_c = np.array(X_c), np.array(y_c)

cw = dict(enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(y_c), y=y_c)))
model_cnn = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(SEQ_LEN, 2)),
    MaxPooling1D(2), Flatten(), Dense(1, activation='sigmoid')
])
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_c[:split], y_c[:split], epochs=15, verbose=0, class_weight=cw)

print("✅ โมเดล LSTM และ CNN ฝึกสอนสำเร็จ")

/Users/hilmanyusoh/Desktop/stock_pattern/venv/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/hilmanyusoh/Desktop/stock_pattern/venv/lib/python3.13/site-packages/keras/src/callbacks/early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
/Users/hilmanyusoh/Desktop/stock_pattern/venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ โมเดล LSTM และ CNN ฝึกสอนสำเร็จ


Cell 5: Sentiment Analysis & Future Forecasting
วิเคราะห์อารมณ์จากข่าวและพยากรณ์ราคาล่วงหน้า

In [51]:
# Sentiment Analysis
sentiment_task = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
news = [f"{SYMBOL} price high", f"Inflation impact on {SYMBOL}", f"Bullish market for {SYMBOL}", f"{SYMBOL} risk warning"]
results = sentiment_task(news)
avg_sentiment = np.mean([res['score'] if res['label'] == 'POSITIVE' else -res['score'] for res in results])

# Future Forecast 50 Days
curr = scaled_data[-SEQ_LEN:].reshape(1, SEQ_LEN, 3)
future_preds = []
for _ in range(FORECAST_DAYS):
    p = model_lstm.predict(curr, verbose=0)[0,0]
    future_preds.append(p)
    curr = np.append(curr[:, 1:, :], [[[p,p,p]]], axis=1)
rescaled_future = inv(np.array(future_preds))

print(f"✅ วิเคราะห์ Sentiment และพยากรณ์อนาคต {FORECAST_DAYS} วันเรียบร้อย")

Device set to use mps:0


✅ วิเคราะห์ Sentiment และพยากรณ์อนาคต 50 วันเรียบร้อย


In [ ]:
## Cell 6: Evaluation (ส่วนที่ต้องใช้ในงานวิจัย)
